In [1]:
import tensorflow as tf
import cv2
import sys
import os
import glob
import random
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
from Misc.MiscUtils import *
from Misc.DataUtils import *
import numpy as np
import time
import argparse
import shutil
from io import StringIO
import string
from termcolor import colored, cprint
import math as m
from tqdm import tqdm
import csv
from sklearn.preprocessing import StandardScaler
# Don't generate pyc codes
sys.dont_write_bytecode = True
# tf.enable_eager_execution()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# def loadData(folder_name, files_in_dir, labels_in_dir, num_training=5000):

#     images_data = []
#     labels_data = []

#     if(len(files_in_dir) < num_training):
#         print("The data has only ", len(files_in_dir) , " images and you are trying to get ",num_training, " images")
#         num_training = len(files_in_dir)

#     for n in range(num_training):
#         image1_name = folder_name + os.sep + "Train_synthetic/PA/" + files_in_dir[n,0]
#         image1 = cv2.imread(image1_name)

#         image2_name = folder_name + os.sep + "Train_synthetic/PB/" + files_in_dir[n,0] 
#         image2 = cv2.imread(image2_name)

#         if(image1 is None) or (image1 is None):
#             print(image1_name, " is empty. Ignoring ...")
#             continue

#         image1 = np.float32(image1)
#         image2 = np.float32(image2)        


#         #combile images along depth

#         image = np.dstack((image1, image1))
#         # #standardize image from 
#         # mean = np.mean(image, axis=(1,2), keepdims=True)
#         # std = np.std(image, axis=(1,2), keepdims=True)
#         # standardized_image = (image - mean) / (std + 0.000001)
         

#         labels_data.append(labels_in_dir[n,:])

#         images_data.append(image)
#         # labels_data.append(label)

#     return np.array(images_data), np.array(labels_data)

In [3]:
# def createDatasetWithBatches(images_data, labels_data, batch_size, shuffle = True):
    
#     N, B = images_data.shape[0], batch_size
#     idx = np.arange(N)
#     if shuffle:
#         np.random.shuffle(idx)
#     return iter((images_data[idx[i:i+B]], labels_data[idx[i:i+B]]) for i in range(0, N, B))


In [4]:
def loadDataBatches(base_path, files_in_dir, labels_in_dir, batch_size):

    images_data = []
    labels_data = []

    for n in range(batch_size):
        random_index = random.randint(0, len(files_in_dir)-1)

        image1_name = base_path + os.sep + "Train_synthetic/PA/" + files_in_dir[random_index, 0]
        image1 = cv2.imread(image1_name)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        
        image2_name = base_path + os.sep + "Train_synthetic/PB/" + files_in_dir[random_index, 0] 
        image2 = cv2.imread(image2_name)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        
        if(image1 is None) or (image1 is None):
            print(image1_name, " is empty. Ignoring ...")
            continue

        image1 = np.float32(image1)
        image2 = np.float32(image2) 

        image = np.dstack((image1, image1))
        images_data.append(image)

        labels_data.append(labels_in_dir[n,:])

    images_data = np.array(images_data)
    labels_data = np.array(labels_data)

    return images_data, labels_data


In [5]:
base_path = "/home/gokul/CMSC733/hgokul_p1/Phase2/Data"
CheckPointPath = "/home/gokul/CMSC733/hgokul_p1/Phase2/Checkpoints/supervised/"
files_in_dir, SaveCheckPoint, ImageSize, NumTrainSamples, labels_in_dir = SetupAll(base_path, CheckPointPath)

In [6]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [7]:
# image_dataset, labels = loadData(BasePath, files_in_dir, labels_in_dir, 100)
# print(image_dataset.shape)
# print(image_dataset.shape)


In [8]:
# dataset_train = createDatasetWithBatches(image_dataset, labels, batch_size = 5, shuffle = True)
# dataset_val = dataset_train

In [9]:
from keras.models import Sequential
from keras.layers import  Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, InputLayer

def ModelInit():

    hidden_layer_size, num_classes = 1000, 8
    input_shape = (128, 128, 2)
    kernel_size = 3
    pool_size = 2
    filters = 64
    dropout = 0.5

    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)
#     layers = [
#         Flatten(input_shape=input_shape),
#         Dense(hidden_layer_size, activation='relu',
#                               kernel_initializer=initializer),
#         Dense(num_classes, activation='softmax', 
#                               kernel_initializer=initializer),
#     ]
#     model = Sequential(layers)
    
    model = Sequential()
    model.add(InputLayer(input_shape))
    ## conv2d 128
    model.add(Conv2D(filters=filters,\
      kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    ## conv2d 128
    model.add(Conv2D(filters=filters,\
            kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size))
    
    ## conv2d 64
    model.add(Conv2D(filters=filters,\
            kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    ## conv2d 64
    model.add(Conv2D(filters=filters,\
            kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size))
    
    ## conv2d 32 2x Filters
    model.add(Conv2D(filters=filters*2,\
            kernel_size=kernel_size, activation='relu', padding='same',))
    model.add(BatchNormalization())
    ## conv2d 32 2x Filters
    model.add(Conv2D(filters=filters*2,\
            kernel_size=kernel_size, activation='relu', padding='same',))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size))
    
    ## conv2d 16 2x Filters
    model.add(Conv2D(filters=filters*2,\
            kernel_size=kernel_size, activation='relu', padding='same',))
    model.add(BatchNormalization())
    ## conv2d 16 2x Filters
    model.add(Conv2D(filters=filters*2,\
            kernel_size=kernel_size, activation='relu', padding='same',))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dropout))
    #for regression model
    model.add(Dense(8))
    return model

def OptimizerInit():
    learning_rate = 0.001
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

Using TensorFlow backend.


In [10]:
def TrainModel(base_path, files_in_dir, labels_in_dir, batch_size, number_of_training_samples, device, model_init_fn, optimizer_init_fn, num_epochs, CheckPointPath, is_training=False): 

    with tf.device(device):

        loss_fn = tf.keras.losses.MeanSquaredError()
        
        model = ModelInit()
        optimizer = OptimizerInit()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.MeanSquaredError(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            num_terations_per_epoch = int(number_of_training_samples / batch_size)

            tf.keras.callbacks.ModelCheckpoint(CheckPointPath, monitor='val_loss', verbose=0, save_best_only=False,
            save_weights_only=False, mode='auto')

            for iteration in tqdm(range(num_terations_per_epoch)):
            
                images_batch, labels_batch = loadDataBatches(base_path, files_in_dir, labels_in_dir, batch_size)

                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(images_batch, training=is_training)
                    loss = loss_fn(labels_batch, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(labels_batch, scores)                    
                    print_every = 10
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()

                        # for test_x, test_y in val_dset:
                        #     # During validation at end of epoch, training set to False
                        #     prediction = model(test_x, training=False)
                        #     t_loss = loss_fn(test_y, prediction)

                        #     val_loss.update_state(t_loss)
                        #     val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result().numpy(),
                                             train_accuracy.result().numpy(),
                                             0,
                                             0))
                    t += 1

In [11]:
batch_size = 64
number_of_training_samples = 1000

In [13]:
TrainModel(base_path, files_in_dir, labels_in_dir, batch_size, number_of_training_samples, device, ModelInit, OptimizerInit, num_epochs=5, CheckPointPath = CheckPointPath, is_training=False)





Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


  0%|          | 0/15 [00:00<?, ?it/s]


ValueError: Layer sequential_1 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[[ 62.,  62.],
         [ 42.,  42.],
         [ 28.,  28.],
         ...,
         [127., 127.],
         [130., 130.],
         [133., 133.]],

        [[ 40.,  40.],
         [ 34.,  34.],
         [ 34.,  34.],
         ...,
         [133., 133.],
         [130., 130.],
         [133., 133.]],

        [[ 43.,  43.],
         [ 32.,  32.],
         [ 40.,  40.],
         ...,
         [119., 119.],
         [114., 114.],
         [135., 135.]],

        ...,

        [[108., 108.],
         [109., 109.],
         [110., 110.],
         ...,
         [101., 101.],
         [ 97.,  97.],
         [ 97.,  97.]],

        [[108., 108.],
         [109., 109.],
         [108., 108.],
         ...,
         [ 95.,  95.],
         [102., 102.],
         [100., 100.]],

        [[109., 109.],
         [108., 108.],
         [110., 110.],
        